# `mammos_entity.io`: reading and writing entities

In [1]:
import mammos_entity as me
import mammos_units as u

## Supported file format

`mammos_entity.io` can read and write csv files containing entity like objects in tabluar format.

The file is structured as follows:
- The first line is commented and contains the preferred ontology label.
- The second line is commented and contains the ontology IRI.
- The third line is commented and contains units.
- The fourth line contains short labels used to refer to individual columns when working with the data, e.g. in a `pandas` dataframe. Omitting spaces in this string is advisable.  
  Ideally this string is the short ontology label.
- All remaining lines contain data.
- Comment lines start with a # as first character. Inline comments are not supported.

If a column has no ontology entry lines 1 and 2 are empty for this column.

If a column has no units (with or without ontology entry) row 3 has no entry for this
column.

Here is an example with four columns:
- an index with no units or ontology label
- spontaneous magnetization from the ontology
- a made-up quantity alpha with a unit but no ontology
- demagnetizing factor with an ontology entry but no unit.

To keep this example short the actual IRIs are omitted:

```
#,SpontaneousMagnetization,,DemagnetizingFactor
#,https://w3id.org/emm/...,,https://w3id.org/emmo/...
#,kA/m,s^2,
index,Ms,alpha,DemagnetizingFactor
0,1e5,1.2,1
1,1e5,3.4,0.5
2,1e5,5.6,0.5
```

## Example

We create some artificial data to write to a csv file.

In [2]:
Ms = me.Ms([1e6, 2e6, 3e6])
T = me.T([1, 2, 3])
theta_angle = [0, 0.5, 0.7] * u.rad
demag_factor = me.Entity("DemagnetizingFactor", [1 / 3, 1 / 3, 1 / 3])
comments = ["Some comment", "Some other comment", "A third comment"]

### Writing
We can write them to a csv file as shown in the following cell. Names of the keyword arguments determine column names in the file.

In [3]:
me.io.entities_to_csv("example.csv", Ms=Ms, T=T, angle=theta_angle, demag_factor=demag_factor, comment=comments)

This has produced the following file:

In [4]:
print(open("example.csv").read())  # noqa: SIM115

#SpontaneousMagnetization,ThermodynamicTemperature,,DemagnetizingFactor,
#https://w3id.org/emmo/domain/magnetic_material#EMMO_032731f8-874d-5efb-9c9d-6dafaa17ef25,https://w3id.org/emmo#EMMO_affe07e4_e9bc_4852_86c6_69e26182a17f,,https://w3id.org/emmo/domain/magnetic_material#EMMO_0f2b5cc9-d00a-5030-8448-99ba6b7dfd1e,
#A / m,K,rad,,
Ms,T,angle,demag_factor,comment
1000000.0,1.0,0.0,0.3333333333333333,Some comment
2000000.0,2.0,0.5,0.3333333333333333,Some other comment
3000000.0,3.0,0.7,0.3333333333333333,A third comment



### Reading
We can read it back in and get a container object (called EntityCollection) containing all columns:

In [5]:
content = me.io.entities_from_csv("example.csv")
content

EntityCollection(
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([1000000., 2000000., 3000000.]), unit='A / m'),
    T=Entity(ontology_label='ThermodynamicTemperature', value=array([1., 2., 3.]), unit='K'),
    angle=<Quantity [0. , 0.5, 0.7] rad>,
    demag_factor=Entity(ontology_label='DemagnetizingFactor', value=array([0.33333333, 0.33333333, 0.33333333])),
    comment=array(['Some comment', 'Some other comment', 'A third comment'],
      dtype=object),
)

We can access the individual elements:

In [6]:
content.Ms

Entity(ontology_label='SpontaneousMagnetization', value=array([1000000., 2000000., 3000000.]), unit='A / m')

In [7]:
content.T

Entity(ontology_label='ThermodynamicTemperature', value=array([1., 2., 3.]), unit='K')

In [8]:
content.angle

<Quantity [0. , 0.5, 0.7] rad>

In [9]:
content.demag_factor

Entity(ontology_label='DemagnetizingFactor', value=array([0.33333333, 0.33333333, 0.33333333]))

In [10]:
content.comment

array(['Some comment', 'Some other comment', 'A third comment'],
      dtype=object)

We can also get a `pandas` dataframe of the data we have read:

In [11]:
content.to_dataframe()

,Ms,T,angle,demag_factor,comment
0,1000000.0,1.0,0.0,0.333333,Some comment
1,2000000.0,2.0,0.5,0.333333,Some other comment
2,3000000.0,3.0,0.7,0.333333,A third comment


### Reading with `pandas`
If we only need the numerical data but not the entity information, we can also read the csv file with pandas:

In [12]:
import pandas as pd

pd.read_csv("example.csv", comment="#")

,Ms,T,angle,demag_factor,comment
0,1000000.0,1.0,0.0,0.333333,Some comment
1,2000000.0,2.0,0.5,0.333333,Some other comment
2,3000000.0,3.0,0.7,0.333333,A third comment


### Check that data has not changed
We can compare with the original data:

In [13]:
Ms == content.Ms

True

In [14]:
T == content.T

True

In [15]:
theta_angle == content.angle

array([ True,  True,  True])

In [16]:
demag_factor == content.demag_factor

True

In [17]:
comments == content.comment

array([ True,  True,  True])